In [1]:
#!/usr/bin/env python

# rwp_core_evaluation.ipynb

In [1]:
from dateutil import rrule
import datetime
from datetime import timedelta
import glob
from netCDF4 import Dataset
import sys, traceback
import dateutil.parser as dparser
import string
from pdb import set_trace as stop
import numpy as np
import numpy.ma as ma
import os
# from mpl_toolkits import basemap
# import ESMF
import pickle
import subprocess
import pandas as pd
from scipy import stats
import copy
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib as mpl
import pylab as plt
import random
import scipy.ndimage as ndimage
import scipy
import shapefile
import matplotlib.path as mplPath
from matplotlib.patches import Polygon as Polygon2
# Cluster specific modules
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import fcluster
from scipy.cluster.vq import kmeans2,vq, whiten
from scipy.ndimage import gaussian_filter
# import seaborn as sns
# import metpy.calc as mpcalc
import shapefile as shp
import sys 
import matplotlib.gridspec as gridspec
import seaborn
# from mpl_toolkits.basemap import Basemap, cm

from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [2]:
from draft_functions import core_2d_properties, core_3d_properties, \
                            watersheding, interpolate_obs, remove_noise

In [3]:
vwp_dir_obs = '/glade/campaign/mmm/c3we/mingge/WRF_DOE/ARM_DATA/RWP/' # SGP_RWP/'

loc = ['SGP', 'MAO']
draft_th = [1.5,3.0,6.0,10.0]
direction = ['up', 'down']
dx_m_all = [4000,2000,1000,500,250,125] 
DT_all = [16,8,4,2,1,0.5]

outdir = '/glade/campaign/mmm/c3we/prein/Papers/2023_Conv-Cores/data/core_stats_rwp/'
if not os.path.exists(outdir):
    os.makedirs(outdir)
    
# speed_ca_median = 15

In [4]:
# observed MCS speed from Dié
date_mao = ['20140319','20140320','20140331','20140401','20140413','20140421','20140509','20140616','20140619','20140703','20140709','20140711','20140712','20140816','20141004','20141015','20141018','20141120','20141127','20141210','20141214','20150109','20150110','20150119','20150303','20150311','20150328','20150329','20150331','20150412','20150425','20150607','20150621','20151106','20151126']
speed_mao = [8.77737,10.660899,8.080038,11.267282,7.2655783,4.816736,5.3585887,5.8819156,11.999847,7.9360313,8.378115,8.336473,8.598592,10.021737,7.2725606,12.406981,8.026962,3.5772574,2.265888,5.3527846,2.7071981,9.480533,9.050027,5.9475303,6.521563,8.723135,5.7211857,4.867622,5.989916,9.14135,7.3281994,6.6606317,11.323382,2.045225,2.8050492]

date_sgp = ['20120531','20140628','20130509','20120615','20140710','20130605','20160308','20130617','20160417','20140602','20160618','20140605','20160729','20140607','20120429','20140612']
speed_sgp = [18.1,19.9,21.8,16.2,15.9,10.3,16.1,17.5,15.3,18.5,19.9,21.8,10.3,21.6,18.5,15.3]

In [8]:
print(np.mean(speed_mao))
print(np.std(speed_mao))

7.274119540000001
2.7070257671852205


In [9]:
print(np.mean(speed_sgp))
print(np.std(speed_sgp))

17.3125
3.4157493687330165


In [7]:
for lo in range(len(loc)):
    files = glob.glob(vwp_dir_obs + loc[lo] + '_ALL/*.cdf')
    for fi in tqdm(range(len(files))):
        
        cores_vp_up_obs = {}
        cores_vp_down_obs = {}
        
        # get the observed movement speed
        if 'sgp' in files[fi]:
            date_act = date_sgp
            speed_act = speed_sgp
        else:
            date_act = date_mao
            speed_act = speed_mao
        sel_case = [date_act[da] == files[fi].split('.')[-3] for da in range(len(date_act))]
        if lo == 0:
            selection = np.array([date_act[da] == files[fi].split('.')[-3] for da in range(len(date_act))])
            if np.sum(selection) == 0:
                print('    no mach found')
                continue
            speed_ca_median = np.array(speed_act)[selection][0]
        else:
            selection = np.array([date_act[da] == files[fi].split('.')[-2] for da in range(len(date_act))])
            if np.sum(selection) == 0:
                print('    no mach found')
                continue
            speed_ca_median = np.array(speed_act)[selection][0]
        
        # # ---------------------------------------------
        # # calculate properties from RWP        
        ncfile = Dataset(files[fi])
        dbz_obs = np.squeeze(ncfile.variables["ReflectivityUAZR"][:]).T
        w_obs = np.squeeze(ncfile.variables["VerticalVelocity"][:]).T
        time_obs = np.squeeze(ncfile.variables["time_offset"][:])
        obs_start = ncfile.variables["time_offset"].units
        height_obs = np.squeeze(ncfile.variables["height"][:])
        ncfile.close()
        
        # create the time vector
        obs_start = datetime.datetime(int(obs_start[14:18]), 
                              int(obs_start[19:21]), 
                              int(obs_start[22:24]),
                              int(obs_start[25:27]),
                              int(obs_start[28:30]))
        obs_dt = np.mean(time_obs[1:] - time_obs[:-1])
        obs_end = obs_start + timedelta(seconds=obs_dt*len(time_obs))
        obs_time = pd.date_range(obs_start, end=obs_end, freq=str(obs_dt)[:1]+'s' + str(obs_dt)[2:4]+'ms')
        
        filled_obs = interpolate_obs(w_obs)
        # set grid cells that are more than two cells appard to nan again
        nan_to_one = np.isnan(w_obs)
        nan_to_one = np.apply_along_axis(lambda m: np.convolve(m, np.ones(5), mode='same'), axis=1, arr=nan_to_one)
        filled_obs[nan_to_one >= 3] = np.nan
        
        filled_obs_clean = remove_noise(filled_obs)
        filled_obs_clean = interpolate_obs(filled_obs_clean)
        filled_obs_clean[nan_to_one >= 3] = np.nan
        
        time_to_length_obs = speed_ca_median * np.array(range(0,len(time_obs)))*6
        
        for dx in range(len(dx_m_all)):
            outfile = outdir + files[fi].split('/')[-1][:-4] +'_'+ str(dx_m_all[dx]) +'m.pkl'
            if os.path.exists(outfile) == False:
                for th in range(len(draft_th)):
                    for di in range(len(direction)):
                        if direction[di] == 'up':
                            dire_sign = 1
                        elif direction[di] == 'down':
                            dire_sign = -1
    
                        # virtual profiler draft characteristics
                        updrafts = np.copy(filled_obs_clean) * dire_sign
                        
                        bins_per_int = int(np.round(dx_m_all[dx]/time_to_length_obs[1],0))
                        intervals = int(updrafts.shape[1]/bins_per_int)
                        updrafts = np.copy(updrafts[:, :intervals * bins_per_int])
                        updrafts = np.mean(np.reshape(updrafts, 
                                                  (updrafts.shape[0], intervals, bins_per_int)), axis = 2)
                        updrafts_th = updrafts > draft_th[th]
                        cloudmask = np.isnan(updrafts)
    
                        # rgiObjectsUD, nr_objectsUD = ndimage.label(updrafts_th,structure = np.ones((3,3)))
                        
                        
                        # # watershedding
                        # coords = peak_local_max(np.array(updrafts), 
                        #                         min_distance = 16,
                        #                         threshold_abs = draft_th[th],
                        #                         labels = updrafts_th
                        #                        )
                        # mask = np.zeros(updrafts.shape, dtype=bool)
                        # mask[tuple(coords.T)] = True
                        # markers, _ = ndi.label(mask)
                        # data_2d_watershed[tt,:,:] = watershed(image = np.array(data[tt,:])*-1,  # watershedding field with maxima transformed to minima
                        #                    markers = markers, # maximum points in 3D matrix
                        #                    connectivity = np.ones((3, 3)), # connectivity
                        #                    offset = (np.ones((2)) * 1).astype('int'), #4000/dx_m[dx]).astype('int'),
                        #                    mask = image, # binary mask for areas to watershed on
                        #                    compactness = 0) # high values --> more regular shaped watersheds
        
                        
                        rgiObjectsUD = watersheding(updrafts, 
                                                     int(2000/dx_m_all[dx]), # int(4000/DT_all[dx]),
                                                     draft_th[th])
                        # filter out cores that are too small
                        min_vol = 10  # grid cells
                        unique_ids, count = np.unique(rgiObjectsUD, return_counts=True)
                        rgiObjectsUD[np.isin(rgiObjectsUD, unique_ids[count < 10])] = 0
                        
                        # sort the objects according to their size
                        Objects=ndimage.find_objects(rgiObjectsUD)                    
                        
                        if direction[di] == 'up':
                            cores_vp_up_obs[str(draft_th[th]).zfill(2)+'_'+files[fi].split('/')[-1][:-4]] = core_2d_properties(Objects,
                                          rgiObjectsUD,
                                          updrafts,
                                          cloudmask,
                                          height_obs*1000,
                                          time_to_length_obs[::bins_per_int][:rgiObjectsUD.shape[1]])
                        elif direction[di] == 'down':
                            cores_vp_down_obs[str(draft_th[th]).zfill(2)+'_'+files[fi].split('/')[-1][:-4]] = core_2d_properties(Objects,
                                          rgiObjectsUD,
                                          updrafts,
                                          cloudmask,
                                          height_obs*1000,
                                          time_to_length_obs[::bins_per_int])
            draft_stats = {
                        "cores_vp_up_obs": cores_vp_up_obs,
                        "cores_vp_down_obs": cores_vp_down_obs,
                        "speed_ca_median": speed_ca_median,
                    }
            
            with open(outfile, 'wb') as handle:
                pickle.dump(draft_stats, handle)

  2%|▏         | 1/42 [00:10<07:06, 10.41s/it]

    no mach found


  7%|▋         | 3/42 [00:20<04:14,  6.53s/it]

    no mach found


 12%|█▏        | 5/42 [00:31<03:36,  5.85s/it]

...

 71%|███████▏  | 30/42 [10:22<04:22, 21.84s/it]

    no mach found


 93%|█████████▎| 39/42 [12:21<00:46, 15.49s/it]

    no mach found


100%|██████████| 42/42 [12:50<00:00, 18.36s/it]


In [ ]:
outdir + files[fi].split('/')[-1][:-4] +'_'+ str(dx_m_all[dx]) +'m.pkl'

In [ ]:
plt.pcolormesh(updrafts, vmin=-10, vmax =10, cmap='coolwarm')
plt.xlim(2400,2550)

In [ ]:
# gr_core_act['mean elevation'] = mean_height
# gr_core_act['mean depth'] = core_height_mean
# gr_core_act['max depth'] = core_heigth_max
# gr_core_act['mean speed'] = core_speed_mean
# gr_core_act['max speed'] = core_speed_max
# gr_core_act['mean width'] = core_with_mean
# gr_core_act['max width'] = core_with_max
# gr_core_act['slope from vertical'] = core_slope

stat = 'slope from vertical'
th = '3.0' # [1.5,3.0,6.0,10.0]
cores_vp = cores_vp_up_obs

# cores_vp = cores_vp_down_obs

mean_width_rwp = [
                    [
                        cores_vp[ii][jj][stat]
                        for jj in cores_vp[ii].keys()
                    ]
                    for ii in [s for s in list(cores_vp.keys()) if th in s]
                ]
mean_width_rwp = np.array([x  for sublist in mean_width_rwp for x in sublist])

In [ ]:
import seaborn as sns

if stat == 'mean elevation':
    newBins = np.arange(0,20000,500)
elif stat == 'mean width':
    newBins = np.arange(0,15000,1000)
elif stat == 'mean speed':
    newBins = np.arange(0,20,1)
elif stat == 'max speed':
    newBins = np.arange(0,20,1)
elif stat == 'slope from vertical':
    newBins = np.arange(-3,3.1,0.1)
    
fig = sns.distplot(mean_width_rwp, color="k", label='RWP', bins=newBins)
# sns.distplot(mean_width_vp, color="r", label='virtual profiler', bins=newBins)
# # sns.distplot( a=mean_width_vp, hist=True, kde=False, rug=False )
# sns.distplot(mean_width_3d, color="b", label='3D', bins=newBins)
plt.xlabel(stat)
plt.legend()
plt.show()

In [ ]:
plt.pcolormesh(time_to_length_obs[::bins_per_int][:rgiObjectsUD.shape[1]]/1000, height_obs,
               updrafts, cmap='coolwarm', vmin=-10, vmax=10)
plt.xlim(400,500)

In [ ]:
plt.pcolormesh(time_to_length_obs[::bins_per_int][:rgiObjectsUD.shape[1]]/1000, height_obs,
               rgiObjectsUD, cmap = plt.cm.nipy_spectral)
plt.xlim(400,500)


In [ ]:
plt.pcolormesh(time_to_length_obs[::bins_per_int][:rgiObjectsUD.shape[1]]/1000, height_obs,
               rgiObjectsUD, cmap = plt.cm.nipy_spectral)
plt.xlim(400,500)


In [ ]:
height_obs

In [ ]:
dx_m_all[dx]

In [ ]:
plt.pcolormesh(filled_obs_clean)

In [ ]:
plt.pcolormesh(updrafts[:,400:1000], cmap='coolwarm', vmin=-10, vmax=10)
plt.contour(rgiObjectsUD[:,400:1000], colors='k', linewidths = 0.1)

In [ ]:
plt.pcolormesh(time_to_length_obs/1000., height_obs,
               filled_obs_clean[:,:], 
               cmap='coolwarm', vmin=-10, vmax=10)
plt.xlim([120,170])
plt.show()


In [ ]:
sim_draft = np.copy(updrafts)
sim_draft[cloudmask!=1] = np.nan

plt.pcolormesh(time_to_length[::bins_per_int][:rgiObjectsUD.shape[1]]/1000, 
               height,
               sim_draft, 
               cmap='coolwarm', vmin=-10, vmax=10)
plt.show()

In [ ]:
cloudmask.shape

In [ ]:
grad_2d[0]

In [ ]:
plt.pcolormesh(w_3D[30,200:300,200:300])

In [ ]:
plt.pcolormesh(rgiObjectsUD)

In [ ]:
plt.pcolormesh(rgiObjectsUD[30,200:300,200:300])

In [ ]:
plt.pcolormesh(rgiObjectsUD[:60,250,:], cmap='coolwarm')

In [ ]:
plt.pcolormesh(w_3D[:60,250,:], cmap='coolwarm', vmin=-20, vmax=20)

In [ ]:
plt.pcolormesh(w_3D[30,:,:], cmap='coolwarm', vmin=-20, vmax = 20)

In [ ]:
plt.pcolormesh(time_to_length/1000., height, data_vp[time_vp,:,VARS.index('WW')].T, vmin=-25, vmax=25, cmap='coolwarm')
plt.contour(time_to_length/1000., height, data_vp[time_vp,:,VARS.index('WW')].T > draft_th[th], colors = 'r')
plt.contour(time_to_length/1000., height, data_vp[time_vp,:,VARS.index('WW')].T < -draft_th[th], colors = 'b')
plt.contour(time_to_length/1000., height, 
               cloudmask[:,:] > 0, 
            colors='k')
plt.show()

In [ ]:
plt.pcolormesh(lon_m/1000., height, w_ca_comz[:,dx_dist,:], vmin=-25, vmax=25, cmap='coolwarm')
plt.contour(lon_m/1000., height, w_ca_comz[:,dx_dist,:] > 3, colors = 'r')
plt.contour(lon_m/1000., height, w_ca_comz[:,dx_dist,:] < -3, colors = 'b')
# plt.contour(lon_m/1000., height, 
#                cloudmask > 0,
#             colors='k')
plt.axhline(y=height[40], color='k', linestyle='-')

In [ ]:
hh = 30
plt.pcolormesh(lat_m/1000., lon_m/1000., w_ca_comz[hh,:,:], vmin=-25, vmax=25, cmap='coolwarm')
# plt.contour(lat_m/1000., lon_m/1000., w_ca_comz[hh,:,:] > 3, colors = 'r')
# plt.contour(lat_m/1000., lon_m/1000., w_ca_comz[hh,:,:] < -3, colors = 'b')
plt.axhline(y=0, color='k', linestyle='-')

### Plot 3D watersheding example

In [ ]:
import numpy as np
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
import matplotlib.pyplot as plt
from scipy import ndimage

tt=30
image = w_ca_comz[:,:,:] > 3
lat_cut = np.argmin(np.abs(lon_m/1000. - 25))

distance = ndimage.distance_transform_edt(image)
local_maxi = peak_local_max(
    distance, footprint=np.ones((3, 3, 3)), labels=np.array(image),
    min_distance=int(4000/dx_m[dx]), threshold_abs=1) # , indices=False

peaks_mask = np.zeros_like(distance, dtype=bool)
peaks_mask.T[local_maxi] = True

markers = ndimage.label(peaks_mask)[0]

labels = watersheding(np.array(image),  # 2D or 3D matrix with labeled objects [np.array]
                 np.array(w_ca_comz[:,:,:]), # vertical wind field
                   int(4000/dx_m[dx]),      # minimum distance between two objects [int]
                   1)


# # markers = ndimage.label(local_maxi)[0]
# labels_orig = watershed(-distance, markers, mask=image)
labels = np.copy(labels.astype(float))
labels[labels == 0] = np.nan


mpl.rcParams.update(mpl.rcParamsDefault)
plt.rcParams.update({'font.size': 15})

print('start plotting')

fig = plt.figure(figsize=(14,15))
# Histogram plots
gs1 = gridspec.GridSpec(3,3) #rgiSize[2])
gs1.update(left=0.06, right=0.97,
           bottom=0.13, top=0.97,
           wspace=0.15, hspace=0.25)

ax = plt.subplot(gs1[0,0])
plt.pcolormesh(lat_m/1000., lon_m/1000., image[tt,:,:]*-1, cmap='gray')
plt.plot([10,40,40,10,10], [40,40,10,10,40], c='r')
plt.plot([0,55], [lon_m[lat_cut]/1000, lon_m[lat_cut]/1000], c='k', ls='--')
plt.title('a) updraft mask')
ax.set_xlabel('Longitude [km]')
ax.set_ylabel('latitude [km]')

ax = plt.subplot(gs1[0,1])
plt.pcolormesh(lat_m/1000., lon_m/1000., w_ca_comz[tt,:,:], vmin=-35, vmax=35, cmap='coolwarm')
plt.plot([10,40,40,10,10], [40,40,10,10,40], c='r')
plt.plot([0,55], [lon_m[lat_cut]/1000, lon_m[lat_cut]/1000], c='k', ls='--')
plt.title('b) vertical wind speed')

ax = plt.subplot(gs1[0,2])
plt.pcolormesh(lat_m/1000., lon_m/1000., labels[tt,:,:], cmap=plt.cm.nipy_spectral, vmin=0, vmax=480)
plt.plot([10,40,40,10,10], [40,40,10,10,40], c='r')
plt.plot([0,55], [lon_m[lat_cut]/1000, lon_m[lat_cut]/1000], c='k', ls='--')
plt.title('c) core segmentation')

# --------------------------------------------
print('    ZOOM INTO CONVECTIVE RETION')
tt1 = tt - 0
ax = plt.subplot(gs1[1,0])
plt.pcolormesh(lat_m/1000., lon_m/1000., image[tt1,:,:]*-1, cmap='gray')
plt.plot([0,55], [lon_m[lat_cut]/1000, lon_m[lat_cut]/1000], c='k', ls='--')
plt.xlim(10, 40)
plt.ylim(10, 40)
plt.title('d) updraft mask zoom in')

ax = plt.subplot(gs1[1,1])
plt.pcolormesh(lat_m/1000., lon_m/1000., w_ca_comz[tt1,:,:], vmin=-35, vmax=35, cmap='coolwarm')
plt.plot([0,55], [lon_m[lat_cut]/1000, lon_m[lat_cut]/1000], c='k', ls='--')
plt.xlim(10, 40)
plt.ylim(10, 40)
plt.contour(lat_m/1000., lon_m/1000., labels[tt1,:,:], cmap=plt.cm.nipy_spectral)
plt.title('e) vertical wind speed zoom in')

ax = plt.subplot(gs1[1,2])
plt.pcolormesh(lat_m/1000., lon_m/1000., labels[tt1,:,:], cmap=plt.cm.nipy_spectral, vmin=0, vmax=480)
plt.plot([0,55], [lon_m[lat_cut]/1000, lon_m[lat_cut]/1000], c='k', ls='--')
plt.xlim(10, 40)
plt.ylim(10, 40)
plt.title('f) core segmentation zoom in')


# --------------------------------------------
print('    VERTICAL CROSS SECTION')


ax = plt.subplot(gs1[2,0])
plt.pcolormesh(lon_m/1000., height/1000, image[:,lat_cut,:]*-1, cmap='gray')
plt.axhline(y=height[tt]/1000, color='k', linestyle='--')
plt.ylim(0, 17)
plt.xlim(0, 55)
ax.set_xlabel('Longitude [km]')
ax.set_ylabel('height [km]')
plt.title('g) updraft mask cross section')

ax = plt.subplot(gs1[2,1])
plt.pcolormesh(lon_m/1000., height/1000, w_ca_comz[:,lat_cut,:], cmap='coolwarm', vmin=-35, vmax=35)
plt.contour(lon_m/1000., height/1000, dbz_ca_comz[:,lat_cut,:], colors = 'k', levels = [10])
plt.axhline(y=height[tt]/1000, color='k', linestyle='--')
plt.ylim(0, 17)
plt.xlim(0, 55)
plt.title('h) vertical wind cross section')

ax = plt.subplot(gs1[2,2])
plt.pcolormesh(lon_m/1000., height/1000, labels[:,lat_cut,:], cmap=plt.cm.nipy_spectral, vmin=0, vmax=480)
plt.contour(lon_m/1000., height/1000, dbz_ca_comz[:,lat_cut,:], colors = 'k', levels = [10])
plt.axhline(y=height[tt]/1000, color='k', linestyle='--')
plt.ylim(0, 17)
plt.xlim(0, 55)
plt.title('i) core segmentation cross section')


plt.show()
# sPlotFile=''
# sPlotName= 'Example-core-segmentation_MCS_'+SIM_All[si]+'.pdf'
# if os.path.isdir(sPlotFile) != 1:
#     subprocess.call(["mkdir","-p",sPlotFile])
# print( '        Plot map to: '+sPlotFile+sPlotName)
# fig.savefig(sPlotFile+sPlotName, bbox_inches='tight') #, dpi=700)

### Does 3D segmentation agree with 2D segmentation?

In [ ]:
crossection_w = w_ca_comz[:,lat_cut,:]
crossection_w_th = crossection_w > 3

rgiObjectsUD, nr_objectsUD = ndimage.label(crossection_w_th,structure=rgiObj_Struct[0,:,:])
# watershedding
rgiObjectsUD = watersheding(rgiObjectsUD, 
                            np.array(crossection_w),
                             int(4000/dx_m[dx]),
                             1)
rgiObjectsUD = np.array(rgiObjectsUD).astype(float)
rgiObjectsUD[rgiObjectsUD == 0] = np.nan

In [ ]:
plt.pcolormesh(lon_m/1000., height, rgiObjectsUD, cmap=plt.cm.nipy_spectral)
plt.axhline(y=height[40], color='k', linestyle='-')
plt.show()

In [ ]:
plt.pcolormesh(lon_m/1000., height, labels[:,lat_cut,:], cmap=plt.cm.nipy_spectral)
plt.axhline(y=height[40], color='k', linestyle='-')
plt.show()

In [ ]:
dx_dist

In [ ]:
core_mean_width

In [ ]:
np.nanmean(np.nansum(core_dx, axis=1)) + np.nanmean(np.nansum(core_dx, axis=2))

In [ ]:
np.nansum(core_dx, axis=1).shape

In [ ]:
core_ouline_act

In [ ]:
# # 3D core properties
# th = 3
# updrafts = w_ca_comz[:,:,:]  * dire_sign > 3 #draft_th[th]
# rgiObjectsUD, nr_objectsUD = ndimage.label(updrafts,structure=rgiObj_Struct[:,:,:])
# # sort the objects according to their size
# Objects=ndimage.find_objects(rgiObjectsUD)

# w_3D = np.copy(w_ca_comz[:,:,:])* dire_sign
# if direction[di] == 'up':
#     cores_3d_up = core_3d_properties(w_3D,
#                       rgiObjectsUD,
#                       Objects,
#                       height,
#                       lat_m,
#                       lon_m)
# elif direction[di] == 'down':
#     cores_3d_down = core_3d_properties(w_3D,
#                       rgiObjectsUD,
#                       Objects,
#                       height,
#                       lat_m,
#                       lon_m)


In [ ]:
# cores_vp[location+'_'+conv_element] = core_2d_properties(Objects,
#                           w_2D,
#                           cloudmask.T,
#                           height,
#                           time_to_length)

In [ ]:
# gr_core_act['mean elevation'] = mean_height
# gr_core_act['mean depth'] = core_height_mean
# gr_core_act['max depth'] = core_heigth_max
# gr_core_act['mean speed'] = core_speed_mean
# gr_core_act['max speed'] = core_speed_max
# gr_core_act['mean width'] = core_with_mean
# gr_core_act['max width'] = core_with_max
# gr_core_act['slope from vertical'] = core_slope

stat = 'mean width'
th = '3.0' # [1.5,3.0,6.0,10.0]
cores_2d = cores_2d_up
cores_vp = cores_vp_up
cores_3d = cores_3d_up



mean_width_2d = np.array( [
                            [
                                cores_2d[ii][jj][stat]
                                for jj in cores_2d[ii].keys()
                            ]
                            for ii in [s for s in list(cores_2d.keys()) if th in s]
                        ])
mean_width_2d = np.array([item for sublist in mean_width_2d for item in sublist])

mean_width_vp = np.array( [
                            [
                                cores_vp[ii][jj][stat]
                                for jj in cores_vp[ii].keys()
                            ]
                            for ii in [s for s in list(cores_vp.keys()) if th in s]
                        ])
mean_width_vp = np.array([item for sublist in mean_width_vp for item in sublist])

mean_width_3d = np.array( [
                            [
                                cores_3d[ii][jj][stat]
                                for jj in cores_3d[ii].keys()
                            ]
                            for ii in [s for s in list(cores_3d.keys()) if th in s]
                        ])
mean_width_3d = np.array([item for sublist in mean_width_3d for item in sublist])

In [ ]:
import seaborn as sns
fig = sns.kdeplot(mean_width_2d, shade=True, color="k", clip=[0, 100000], label='2D')
fig = sns.kdeplot(mean_width_vp, shade=True, color="r", clip=[0, 100000], label='virtual profiler')
fig = sns.kdeplot(mean_width_3d, shade=True, color="b", clip=[0, 100000], label='3D')
plt.legend()
plt.show()

In [ ]:
mean_width_vp

In [ ]:
# virtual profiler draft characteristics
updrafts = DATA[time_vp,:,loc,VARS.index('WW')].T > 10
rgiObjectsUD, nr_objectsUD = ndimage.label(updrafts,structure=rgiObj_Struct[0,:,:])

# sort the objects according to their size
Objects=ndimage.find_objects(rgiObjectsUD)
# rgiVolObj=np.array([np.sum(rgiObjectsUD[Objects[ob]] == ob+1) for ob in range(nr_objectsUD)])

In [ ]:

w_2D = np.copy(DATA[time_vp,:,loc,VARS.index('WW')].T)
gr_core = core_2d_properties(Objects,
                  w_2D,
                  cloudmask.T,
                  height,
                  time_to_length)

In [ ]:
gr_core

In [ ]:
plt.pcolormesh(lat_m / 1000., height,
               dbz_ca[:,dx_dist,:], vmin=0, vmax=60, cmap='coolwarm')
# plt.axvline(x=lon_vs_id[ii], c='k')

In [ ]:
plt.pcolormesh(lat_m / 1000., height,
               w_ca_comz[:,dx_dist,:], vmin=-25, vmax=25, cmap='coolwarm')
# plt.axvline(x=lon_vs_id[ii], c='k')

In [ ]:
plt.pcolormesh(lat_m / 1000.,
               lon_m / 1000.,
               w_ca[35,:,:], vmin=-25, vmax=25, cmap='coolwarm')
plt.contour(lat_m / 1000.,
               lon_m / 1000.,
               w_ca[35,:,:], levels=[3], colors=['k'], linewidths = 0.5)
plt.plot(0, 0, 'ok')

In [ ]:
plt.pcolormesh(lat_m / 1000.,
               lon_m / 1000.,
               dbz_ca[35,:,:], vmin=0, vmax=60, cmap='coolwarm')
plt.plot(0, 0, 'ok')

In [ ]:
files_int[fi]

In [ ]:
plt.pcolormesh(lon,lat,dbz[10,:,:]>42, vmin=0, vmax=1, cmap = 'coolwarm')
for ii in tqdm(range(len(lat_vs))):
    plt.plot(lon_vs[ii], lat_vs[ii], 'ok')

In [ ]:
plt.pcolormesh(lon,lat,dbz_smooth[15,:,:]>42, vmin=0, vmax=1, cmap = 'coolwarm')
plt.plot(lon_vs[8], lat_vs[8], 'ok')

In [ ]:
plt.pcolormesh(lon,lat,MS_objects[14,:,:], vmin=0, vmax=25, cmap = 'coolwarm')
plt.plot(lon_vs[8], lat_vs[8], 'ok')

In [ ]:
plt.pcolormesh(lon,lat,MS_objects[19,:,:], vmin=0, vmax=10, cmap = 'coolwarm')
plt.plot(lon_vs[8], lat_vs[8], 'ok')